In [1]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)




using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
--- startup: 8.724 seconds ---


In [ ]:
import re

def clean_gutenberg_text(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 2. Remove page numbers or headers like [Page 1]
    text = re.sub(r'\[Page \d+\]', '', text)

    # 3. Normalize whitespace
    # Remove extra spaces
    text = re.sub(r'[ \t]+', ' ', text)
    # Merge broken lines (assume that a line ending with a lowercase letter continues)
    text = re.sub(r'(?<=[a-z,;])\n(?=[a-zA-Z])', ' ', text)
    # Normalize multiple blank lines to exactly two (paragraph break)
    text = re.sub(r'\n{2,}', '\n\n', text)
    # Strip leading/trailing whitespace
    text = text.strip()

    # 4. Fix Unicode characters (basic replacements)
    text = text.replace('“', '"').replace('”', '"').replace('‘', "'").replace('’', "'")
    text = text.replace('—', '-').replace('–', '-')  # em dash and en dash to hyphen
    text = text.replace('\r', '')  # Remove carriage returns if any
    text = text.replace('_','')

    # 6. Write cleaned text
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f"Cleaned text saved to {output_path}")


clean_gutenberg_text(
    input_path='winnie_the_pooh.txt',
    output_path='winnie_the_pooh_cleaned.txt',
)

Cleaned text saved to winnie_the_pooh_cleaned.txt


In [2]:
# Input file to process
input_file="winnie_the_pooh.txt"

# Output directory to store resulting files in
output_directory="./output_dir/winnie/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="winnie_the_pooh"

booknlp.process(input_file, output_directory, book_id)


--- spacy: 3.149 seconds ---
--- entities: 18.388 seconds ---
--- quotes: 0.018 seconds ---
Cannot resolve quotation
--- attribution: 65.066 seconds ---
--- name coref: 0.020 seconds ---
--- coref: 21.408 seconds ---
--- TOTAL (excl. startup): 108.098 seconds ---, 34786 words


In [3]:
import json
from collections import Counter

def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

data=proc("./output_dir/winnie/winnie_the_pooh.book")

In [4]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

for character in data["characters"]:

    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]

    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]

        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=100
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))
        print()

44 967 Pooh he/him/his

	poss	9 head
	poss	8 house
	poss	7 nose
	poss	6 legs
	poss	5 umbrella
	poss	4 tree
	poss	4 boat
	poss	3 eyes
	poss	3 friend
	poss	3 mouth
	poss	3 branch
	poss	2 gun
	poss	2 bath
	poss	2 paw
	poss	2 honey
	poss	2 tongue
	poss	2 jar
	poss	2 door
	poss	2 chair
	poss	2 feet
	poss	1 face
	poss	1 Bear
	poss	1 house--_do
	poss	1 paws
	poss	1 Rabbit
	poss	1 friends
	poss	1 walk
	poss	1 idea
	poss	1 uncle
	poss	1 preparations
	poss	1 shoes
	poss	1 hand
	poss	1 family
	poss	1 joy
	poss	1 Boots
	poss	1 back
	poss	1 ,
	poss	1 boot
	poss	1 song
	poss	1 whisper
	poss	1 place
	poss	1 lunch
	poss	1 hands
	poss	1 handkerchief
	poss	1 hardest
	poss	1 pole
	poss	1 mistake
	poss	1 tail
	poss	1 way
	poss	1 supper
	poss	1 pot
	poss	1 stick
	poss	1 life
	poss	1 mind
	poss	1 spoon
	poss	1 bear

	agent	148 said
	agent	23 thought
	agent	21 looked
	agent	19 went
	agent	18 got
	agent	17 see
	agent	16 know
	agent	16 think
	agent	13 had
	agent	10 took
	agent	9 have
	agent	8 tried
	agent	8 ca

In [3]:
import pandas as pd

df_tokens = pd.read_csv("./output_dir/princess/princess.tokens", delimiter="\t")
df_entities = pd.read_csv("./output_dir/princess/princess.entities", delimiter="\t")

#create dictionary with the shape [sentence_ID][token_ID_within_sentence][word]
sentence_and_token_ID_to_word_dict = {}
for character in data["characters"]:
    character_id=character["id"]
    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
    else:
        max_proper_mention = character_id

    unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
    #print(unique_start_token_ids)
    sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
    sentence_ids = sentence_ids.reset_index(drop=True)

    for index, row in sentence_ids.iterrows():
        sentence_ID = row['sentence_ID']
        token_ID = row['token_ID_within_sentence']
        combined_sentence_and_token_ID = str(sentence_ID) +":"+ str(token_ID)
        sentence_and_token_ID_to_word_dict[combined_sentence_and_token_ID] = max_proper_mention
    

In [29]:
#the number sets which character we are looking at
character = data["characters"][10]
character_id=character["id"]

#get all sentencs where COREF = character_id
unique_start_token_ids = df_entities.loc[df_entities['COREF'] == character_id, 'start_token']
print(unique_start_token_ids)
sentence_ids = df_tokens[df_tokens['token_ID_within_document'].isin(unique_start_token_ids)][['token_ID_within_document', 'token_ID_within_sentence', 'sentence_ID']]
sentence_ids = sentence_ids.reset_index(drop=True)
print(sentence_ids)

38         226
64         381
239       1267
240       1271
241       1280
         ...  
12014    76194
12015    76197
12080    76570
12393    78306
12394    78314
Name: start_token, Length: 144, dtype: int64
     token_ID_within_document  token_ID_within_sentence  sentence_ID
0                         226                        18            6
1                         381                         0           12
2                        1267                         9           48
3                        1271                        13           48
4                        1280                         1           49
..                        ...                       ...          ...
139                     76194                         3         4222
140                     76197                         6         4222
141                     76570                         2         4240
142                     78306                         1         4331
143                     78314  

In [5]:
from allennlp.predictors.predictor import Predictor
SRL_MODEL_PATH = "srl.tar.gz"
predictor_srl = Predictor.from_path(SRL_MODEL_PATH)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import utils
import spacy
nlp = spacy.load("en_core_web_sm")

In [21]:
def check_for_arg(n, arg_name, dict_name, sentence_ID):
    if arg_name in n["tags"]:
        index = n['tags'].index(arg_name)
        if str(sentence_ID)+":"+str(index) in dict_name.keys():
            return dict_name[str(sentence_ID)+":"+str(index)]
        else:
            return None
    else:
        return None
    


In [30]:
columns = ['sentence_ID', 'verb', 'agent', 'patient']
srl_results_df = pd.DataFrame(columns=columns)

# for each mention in our coreference cluster do the following
for index, row in sentence_ids.iterrows():
    token_ID_within_document = row['token_ID_within_document']
    #word = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'word'].values[0]

    sentence_ID = df_tokens.loc[df_tokens['token_ID_within_document'] == token_ID_within_document, 'sentence_ID'].values[0]
    words_list = df_tokens.loc[df_tokens['sentence_ID'] == sentence_ID, 'word'].tolist()
    token_ID_within_sentence = row['token_ID_within_sentence']
    try:
        sentence_for_srl = ' '.join(words_list)
    except:
        print("Error: ", words_list)
        continue
    predictions_srl = predictor_srl.predict(sentence_for_srl)

    for n in predictions_srl['verbs']:
        if token_ID_within_sentence < len(n['tags']):
            tag = n['tags'][token_ID_within_sentence]
            if "B-ARG0" in tag:

                word = check_for_arg(n, "B-ARG1", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': ["protag"], 'patient': [word]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)
            elif "B-ARG1" in tag:
                word = check_for_arg(n, "B-ARG0", sentence_and_token_ID_to_word_dict, sentence_ID)
                if word != None:
                    new_row = pd.DataFrame({'sentence_ID': [sentence_ID], 'verb': [utils.lemmatize(n['verb'], nlp)], 'agent': [word], 'patient': ["protag"]})
                    srl_results_df = pd.concat([srl_results_df, new_row], ignore_index=True)

srl_results_df.to_csv('srl_output_lavinia_little_princess.csv', index=False)
del srl_results_df